# M1M3 actuator movies
Craig Lage - 07-Jan-24 \
This code plots the M1M3 force actuator errors during a slew.

In [ ]:
import sys, time, os, asyncio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

## Set up the necessary subroutines

In [ ]:
def hardPointPlot(df, ax):
    ax.set_title("Hardpoint forces")
    ax.set_ylabel("measuredForce(N)")
    ax.set_ylim(-3500, 3500)
    times = df['timestamp'].values
    for i in range(6):
        data = df[f'measuredForce{i}'].values
        ax.plot(times, data, label=f'HP{i}')
    ax.set_xticks([])    
    ax.plot([times[0], times[-1]], [3000, 3000], color='red')
    ax.plot([times[0], times[-1]], [-3000, -3000], color='red')
    ax.plot([times[0], times[-1]], [1000, 1000], ls='--', color='blue')
    ax.plot([times[0], times[-1]], [-1000, -1000], ls='--', color='blue')
    ax.legend(loc='upper left', fontsize=8)
    return

def TMAPlots(plotAz, plotEl, axP, axV, axA, axJ):
    smoothingFactor = 0.40 # In spline creation
    smoothingFactor2 = 0.0
    kernelSize = 5 # In convolution
    kernel = np.ones(kernelSize) / kernelSize
    # Now calculates the spline fit and differentiate it to get the acceleration and jerk
    azPs = plotAz['actualPosition'].values
    azVs = plotAz['actualVelocity'].values
    azXs = plotAz['timestamp'].values - plotAz['timestamp'].values[0]  
    elPs = plotEl['actualPosition'].values
    elVs = plotEl['actualVelocity'].values
    elXs = plotEl['timestamp'].values - plotEl['timestamp'].values[0]
    plotStart = azXs[0] + 1.0
    plotEnd = azXs[-1] - 1.0
    
    plotAzXs = np.linspace(azXs[0], azXs[-1], 2000)
    plotElXs = np.linspace(elXs[0], elXs[-1], 2000)
    azPSpline = UnivariateSpline(azXs, azPs, s=0)
    azVelSpline1 = UnivariateSpline(azXs, azVs, s=smoothingFactor) 
    # Now smooth the derivative before differentiating again
    smoothedAzVel = np.convolve(azVelSpline1(plotAzXs), kernel, mode='same')
    azVelSpline = UnivariateSpline(plotAzXs, smoothedAzVel, s=smoothingFactor2)
    azAccSpline1 = azVelSpline.derivative(n=1)
    smoothedAzAcc = np.convolve(azAccSpline1(plotAzXs), kernel, mode='same')
    # Now smooth the derivative before differentiating again
    azAccSpline = UnivariateSpline(plotAzXs, smoothedAzAcc, s=smoothingFactor2)
    azJerkSpline = azAccSpline.derivative(n=1) 
    elPSpline = UnivariateSpline(elXs, elPs, s=0)
    elVelSpline1 = UnivariateSpline(elXs, elVs, s=smoothingFactor)
    # Now smooth the derivative before differentiating again
    smoothedElVel = np.convolve(elVelSpline1(plotElXs), kernel, mode='same')
    elVelSpline = UnivariateSpline(plotElXs, smoothedElVel, s=smoothingFactor2)
    elAccSpline1 = elVelSpline.derivative(n=1)
    smoothedElAcc = np.convolve(elAccSpline1(plotElXs), kernel, mode='same')                
    # Now smooth the derivative before differentiating again
    elAccSpline = UnivariateSpline(plotElXs, smoothedElAcc, s=smoothingFactor2)
    elJerkSpline = elAccSpline.derivative(n=1) 
    maxAzAccel = np.max(abs(azAccSpline(plotAzXs)))
    maxAzJerk = np.max(abs(azJerkSpline(plotAzXs)))
    maxElAccel = np.max(abs(elAccSpline(plotAzXs)))
    maxElJerk = np.max(abs(elJerkSpline(plotAzXs)))

    axP.plot(plotAzXs, azPSpline(plotAzXs), lw=3, color='r')
    axP.scatter(azXs, azPs, marker='x', color='r', s=100)
    axP.set_ylabel("Azimuth (deg)", color='r')
    axP.set_xticks([])
    axP2 = axP.twinx()
    axP2.plot(plotElXs, elPSpline(plotElXs), lw=3, color='g')
    axP2.scatter(elXs, elPs, marker='x', color='g', s=100)
    axP2.set_ylabel("Elevation (deg)", color='g')

    axV.plot(plotAzXs, azVelSpline(plotAzXs), lw=3, color='r')
    axV.scatter(azXs, azVs, marker='x', color='r', s=100)
    axV.set_ylabel("Azimuth Vel(deg/sec)", color='r')
    axV.set_xticks([])
    axV2 = axV.twinx()
    axV2.plot(plotElXs, elVelSpline(plotElXs), lw=3, color='g')
    axV2.scatter(elXs, elVs, marker='x', color='g', s=100)
    axV2.set_ylabel("Elevation Vel(deg/sec)", color='g')

    axA.plot(plotAzXs, azAccSpline(plotAzXs), lw=3, color='r', label=f"Max Acc={maxAzAccel:.1f}")
    axA.set_ylabel("Az Accel(deg/sec^2)", color='r')
    axA.set_xticks([])
    axA.legend(loc='upper left')
    axA2 = axA.twinx()
    axA2.plot(plotElXs, elAccSpline(plotElXs), lw=3, color='g', label=f"Max Acc={maxElAccel:.1f}")
    axA2.set_ylabel("El Accel(deg/sec^2)", color='g')
    axA2.legend(loc='upper right')
    
    axJ.plot(plotAzXs, azJerkSpline(plotAzXs), lw=3, color='r', label=f"Max Jerk={maxAzJerk:.1f}")
    axJ.set_ylabel("Az Jerk(deg/sec^3)", color='r')
    axJ.legend(loc='upper left')
    axJ2 = axJ.twinx()
    axJ2.plot(plotElXs, elJerkSpline(plotElXs), lw=3, color='g', label=f"Max Jerk={maxElJerk:.1f}")
    axJ2.set_ylabel("El Jerk(deg/sec^3)", color='g')
    axJ2.legend(loc='upper right')
    axJ.set_xlabel("Time(sec)")

    return


## Now generate the plots

In [ ]:
%matplotlib inline
client = EfdClient('usdf_efd')
dayObs = 20240102
seqNum = 1294
dayObs = 20240109
seqNum = 147
dayObs = 20240104
seqNum = 325

eventMaker = TMAEventMaker()
event = eventMaker.getEvent(dayObs, seqNum)
start = event.begin
end = event.end

padStart = 2.0
plotStart = start - TimeDelta(padStart, format='sec') 
padEnd = 2.0
plotEnd = end + TimeDelta(padEnd, format='sec') 
hardpoints = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", "*", plotStart, plotEnd)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['*'],  plotStart, plotEnd)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['*'],  plotStart, plotEnd) 

fig = plt.figure(figsize=(8,12))
fig.suptitle(f"Hardpoints and Vel/Acc/Jerks {dayObs} - seqNum {seqNum}", y=0.90)
axHP = fig.add_axes((0.1, 0.70, 0.8, 0.15))
hardPointPlot(hardpoints, axHP)
axP = fig.add_axes((0.1, 0.55, 0.8, 0.15))
axV = fig.add_axes((0.1, 0.40, 0.8, 0.15))
axA = fig.add_axes((0.1, 0.25, 0.8, 0.15))
axJ = fig.add_axes((0.1, 0.10, 0.8, 0.15))
TMAPlots(az, el, axP, axV, axA, axJ)
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/HP_VelAccJerk_Plots_{dayObs}_{seqNum}.png")